# 🚀 Enhanced ML Model Training with Comprehensive Visualization & Reporting

This notebook now includes advanced visualization and automated reporting capabilities while maintaining all existing model training and prediction logic.

## Key Enhancements:
- **Model Performance Dashboards**: Comprehensive comparison charts and metrics visualization
- **Feature Importance Analysis**: Interactive feature importance plots and correlation heatmaps
- **Prediction Analysis Suite**: Detailed prediction vs actual analysis with error distributions
- **Automated Reporting**: Professional HTML reports with executive summaries
- **Preserved Model Logic**: All existing ensemble learning and evaluation logic remains intact

Let's proceed with the enhanced model training and analysis...

In [ ]:
import numpy as np
import pandas as pd
import gc
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import TimeSeriesSplit

import xgboost as xgb
import lightgbm as lgb

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tools.sm_exceptions import ConvergenceWarning

try:
    import matplotlib.pyplot as plt
except ImportError:
    print("Warning: matplotlib not available, plotting functions will be disabled")
    plt = None

from app_config import Config
from model_utils import ModelDataProcessor, ModelEvaluator, ModelManager

# 🎨 NEW: Import comprehensive visualization and reporting utilities
from visualization_utils import ComprehensiveVisualizer
from automated_reporting import AutomatedReportGenerator

# Initialize secure configuration and utilities
config = Config()
data_processor = ModelDataProcessor()
evaluator = ModelEvaluator()
model_manager = ModelManager()

# 🎨 NEW: Initialize visualization and reporting tools
visualizer = ComprehensiveVisualizer(config)
report_generator = AutomatedReportGenerator(config)

# Mount Google Drive if in Colab environment
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive mounted")
except ImportError:
    print("ℹ️ Not in Colab environment, skipping Google Drive mount")

print("✅ All modules and enhanced utilities loaded successfully!")
print("🎨 Comprehensive visualization and reporting capabilities enabled!")

In [ ]:
# =====================================
# 📊 ML Data Validation Framework
# =====================================

def validate_ml_dataframe(df: pd.DataFrame, required_columns: List[str] = None, 
                         target_column: str = 'Close') -> pd.DataFrame:
    """
    Validate DataFrame for ML training with comprehensive checks.
    
    Args:
        df: Input DataFrame
        required_columns: List of required column names
        target_column: Name of target column
    
    Returns:
        Validated DataFrame ready for ML processing
    """
    if df is None or df.empty:
        raise ValueError("Cannot train models on empty DataFrame")
    
    print(f"🔍 Starting ML data validation for {len(df)} records...")
    
    # Basic structure validation
    if len(df) < 100:
        print(f"⚠️ Warning: Dataset has only {len(df)} records, which may be insufficient for training")
    
    # Check for target column
    if target_column not in df.columns:
        # Try to find suitable target
        close_cols = [col for col in df.columns if 'close' in col.lower()]
        if close_cols:
            target_column = close_cols[0]
            print(f"🎯 Using {target_column} as target column")
        else:
            raise ValueError(f"Target column '{target_column}' not found and no suitable alternative")
    
    # Validate target column
    if df[target_column].isna().all():
        raise ValueError(f"Target column '{target_column}' contains only NaN values")
    
    # Check for sufficient non-NaN target values
    valid_target_count = df[target_column].notna().sum()
    if valid_target_count < len(df) * 0.7:  # Less than 70% valid targets
        print(f"⚠️ Warning: Only {valid_target_count}/{len(df)} ({valid_target_count/len(df)*100:.1f}%) target values are valid")
    
    # Validate required columns if specified
    if required_columns:
        missing_cols = [col for col in required_columns if col not in df.columns]
        if missing_cols:
            print(f"⚠️ Missing required columns: {missing_cols}")
            # Add missing columns with NaN
            for col in missing_cols:
                df[col] = np.nan
    
    # Remove columns with all NaN values
    all_nan_cols = df.columns[df.isna().all()].tolist()
    if all_nan_cols:
        print(f"Removing {len(all_nan_cols)} columns with all NaN values")
        df = df.drop(columns=all_nan_cols)
    
    # Remove columns with single unique value (excluding NaN)
    single_value_cols = []
    for col in df.select_dtypes(include=[np.number]).columns:
        unique_vals = df[col].dropna().nunique()
        if unique_vals <= 1:
            single_value_cols.append(col)
    
    if single_value_cols and target_column not in single_value_cols:
        print(f"Removing {len(single_value_cols)} columns with single values: {single_value_cols[:5]}{'...' if len(single_value_cols) > 5 else ''}")
        df = df.drop(columns=single_value_cols)
    
    # Handle infinite values
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    inf_counts = {}
    for col in numeric_cols:
        inf_count = np.isinf(df[col]).sum()
        if inf_count > 0:
            inf_counts[col] = inf_count
            df[col] = df[col].replace([np.inf, -np.inf], np.nan)
    
    if inf_counts:
        print(f"Replaced infinite values in {len(inf_counts)} columns")
    
    # Check for extreme outliers in target
    target_mean = df[target_column].mean()
    target_std = df[target_column].std()
    
    if target_std > 0:
        z_scores = np.abs((df[target_column] - target_mean) / target_std)
        extreme_outliers = (z_scores > 10).sum()
        
        if extreme_outliers > 0:
            print(f"⚠️ Found {extreme_outliers} extreme outliers in target column (>10 std dev)")
            # Cap outliers instead of removing them
            outlier_mask = z_scores > 10
            df.loc[outlier_mask, target_column] = target_mean + (10 * target_std * np.sign(df.loc[outlier_mask, target_column] - target_mean))
    
    print(f"✅ ML data validation completed: {len(df)} records, {len(df.columns)} features")
    return df

def validate_ml_features(X, y, feature_names=None):
    """
    Validate feature matrix and target vector for ML training.
    
    Args:
        X: Feature matrix
        y: Target vector
        feature_names: List of feature names (optional)
    
    Returns:
        Validated X, y, and feature_names
    """
    print(f"🔍 Validating ML features: X{X.shape}, y{y.shape if hasattr(y, 'shape') else len(y)}")
    
    # Convert to numpy if needed
    if hasattr(X, 'values'):
        X = X.values
    if hasattr(y, 'values'):
        y = y.values
    
    X = np.array(X)
    y = np.array(y)
    
    # Basic shape validation
    if len(X) != len(y):
        raise ValueError(f"Feature matrix length ({len(X)}) doesn't match target length ({len(y)})")
    
    if len(X) == 0:
        raise ValueError("Empty feature matrix")
    
    # Check for NaN values
    nan_features = np.isnan(X).any(axis=0)
    nan_target = np.isnan(y)
    
    if nan_features.any():
        nan_feature_count = nan_features.sum()
        print(f"⚠️ Found NaN values in {nan_feature_count} features")
        
        # Remove features that are mostly NaN
        nan_ratio = np.isnan(X).mean(axis=0)
        high_nan_features = nan_ratio > 0.5
        
        if high_nan_features.any():
            print(f"Removing {high_nan_features.sum()} features with >50% NaN values")
            X = X[:, ~high_nan_features]
            if feature_names:
                feature_names = [name for i, name in enumerate(feature_names) if not high_nan_features[i]]
        
        # Impute remaining NaN values
        from sklearn.impute import SimpleImputer
        imputer = SimpleImputer(strategy='median')
        X = imputer.fit_transform(X)
    
    if nan_target.any():
        valid_mask = ~nan_target
        print(f"Removing {nan_target.sum()} samples with NaN target values")
        X = X[valid_mask]
        y = y[valid_mask]
    
    # Check for constant features
    feature_std = np.std(X, axis=0)
    constant_features = feature_std == 0
    
    if constant_features.any():
        print(f"Removing {constant_features.sum()} constant features")
        X = X[:, ~constant_features]
        if feature_names:
            feature_names = [name for i, name in enumerate(feature_names) if not constant_features[i]]
    
    # Final validation
    if X.shape[1] == 0:
        raise ValueError("No valid features remaining after validation")
    
    print(f"✅ Feature validation completed: X{X.shape}, y{y.shape}")
    return X, y, feature_names

def safe_model_training(model_func, X, y, *args, **kwargs):
    """
    Safely train a model with error handling and validation.
    
    Args:
        model_func: Function to train the model
        X: Feature matrix
        y: Target vector
        *args, **kwargs: Additional arguments for model_func
    
    Returns:
        Trained model or None if training fails
    """
    try:
        # Validate inputs
        X, y, _ = validate_ml_features(X, y)
        
        if len(X) < 50:
            print(f"⚠️ Warning: Training with only {len(X)} samples")
        
        # Train model
        model = model_func(X, y, *args, **kwargs)
        print(f"✅ Model training completed successfully")
        return model
        
    except Exception as e:
        print(f"❌ Model training failed: {str(e)}")
        return None

print("✅ ML data validation utilities loaded successfully!")


In [ ]:
# =====================================
# 🎨 COMPREHENSIVE MODEL VISUALIZATION & REPORTING
# =====================================

print("🎨 Starting comprehensive model visualization and reporting...")

# Run the main training function first to get results
try:
    # Execute the main training pipeline
    predictor, ensemble_predictions, test_targets, results_df, individual_metrics = main_with_analysis()
    
    print("\n" + "="*60)
    print("🎨 GENERATING COMPREHENSIVE VISUALIZATIONS")
    print("="*60)
    
    # =====================================
    # 📊 Model Performance Dashboard
    # =====================================
    
    print("📊 Creating model performance dashboard...")
    
    # Create comprehensive model performance dashboard
    dashboard_path = visualizer.create_model_performance_dashboard(
        individual_metrics,
        save_name="ml_model_performance_dashboard"
    )
    
    if dashboard_path:
        print(f"✅ Model performance dashboard saved: {dashboard_path}")
    else:
        print("⚠️ Dashboard creation skipped (plotting not available)")
    
    # =====================================
    # 🎯 Prediction Analysis Suite
    # =====================================
    
    print("🎯 Creating prediction analysis suite...")
    
    # Create comprehensive prediction analysis
    prediction_suite_path = visualizer.create_prediction_analysis_suite(
        test_targets, 
        ensemble_predictions,
        {model_name: pred[-len(test_targets):] for model_name, pred in predictor.predictions.items() if len(pred) >= len(test_targets)},
        save_name="ml_prediction_analysis_suite"
    )
    
    if prediction_suite_path:
        print(f"✅ Prediction analysis suite saved: {prediction_suite_path}")
    
    # =====================================
    # 🧠 Feature Importance Analysis
    # =====================================
    
    print("🧠 Creating feature importance analysis...")
    
    # Get feature names from the predictor
    if hasattr(predictor, 'tree_model_features') and predictor.tree_model_features:
        feature_names = predictor.tree_model_features
    else:
        feature_names = [f"Feature_{i}" for i in range(50)]  # Default feature names
    
    # Extract models that have feature importance
    models_with_importance = {}
    for model_name, model in predictor.models.items():
        if hasattr(model, 'feature_importances_') or hasattr(model, 'coef_'):
            models_with_importance[model_name] = model
    
    if models_with_importance:
        feature_analysis_path = visualizer.create_feature_importance_analysis(
            models_with_importance,
            feature_names,
            save_name="ml_feature_importance_analysis"
        )
        
        if feature_analysis_path:
            print(f"✅ Feature importance analysis saved: {feature_analysis_path}")
    else:
        print("⚠️ No models with feature importance available")
    
    # =====================================
    # 📋 Comprehensive Model Report
    # =====================================
    
    print("📋 Generating comprehensive model analysis report...")
    
    try:
        # Prepare training data for report (use a sample if too large)
        if hasattr(predictor, 'train_data') and predictor.train_data is not None:
            sample_size = min(1000, len(predictor.train_data))  # Limit for performance
            training_sample = predictor.train_data.sample(n=sample_size) if len(predictor.train_data) > sample_size else predictor.train_data
        else:
            # Create minimal training data representation
            training_sample = pd.DataFrame({'feature_1': test_targets, 'target': test_targets})
        
        # Generate comprehensive report
        comprehensive_report_path = report_generator.generate_comprehensive_model_report(
            models_dict=predictor.models,
            results_dict=individual_metrics,
            predictions_dict=predictor.predictions,
            y_true=test_targets,
            y_pred_ensemble=ensemble_predictions,
            training_data=training_sample,
            feature_names=feature_names,
            report_name="comprehensive_ml_model_analysis"
        )
        
        if comprehensive_report_path:
            print(f"✅ Comprehensive model report saved: {comprehensive_report_path}")
        else:
            print("⚠️ Comprehensive report generation failed")
            
    except Exception as e:
        print(f"⚠️ Comprehensive report generation failed: {str(e)}")
        
        # Generate simplified prediction performance report as fallback
        try:
            simplified_report_path = report_generator.generate_prediction_performance_report(
                test_targets,
                ensemble_predictions,
                model_name="Ensemble_Model"
            )
            
            if simplified_report_path:
                print(f"✅ Simplified prediction report saved: {simplified_report_path}")
                
        except Exception as e2:
            print(f"⚠️ Simplified report also failed: {str(e2)}")
    
    # =====================================
    # 📈 Model Performance Summary with Visualizations
    # =====================================
    
    print("\n" + "="*60)
    print("📈 ENHANCED PERFORMANCE SUMMARY")
    print("="*60)
    
    # Enhanced performance summary
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    
    # Calculate comprehensive ensemble metrics
    ensemble_mse = mean_squared_error(test_targets, ensemble_predictions)
    ensemble_rmse = np.sqrt(ensemble_mse)
    ensemble_mae = mean_absolute_error(test_targets, ensemble_predictions)
    ensemble_r2 = r2_score(test_targets, ensemble_predictions)
    ensemble_mape = np.mean(np.abs((test_targets - ensemble_predictions) / test_targets)) * 100
    
    print(f"🎯 ENSEMBLE PERFORMANCE:")
    print(f"   MSE: {ensemble_mse:.8f}")
    print(f"   RMSE: {ensemble_rmse:.8f}")
    print(f"   MAE: {ensemble_mae:.8f}")
    print(f"   R²: {ensemble_r2:.8f}")
    print(f"   MAPE: {ensemble_mape:.4f}%")
    
    # Best individual model
    if individual_metrics:
        best_model = min(individual_metrics.items(), key=lambda x: x[1].get('RMSE', float('inf')))
        print(f"\n🏆 BEST INDIVIDUAL MODEL: {best_model[0]}")
        print(f"   RMSE: {best_model[1].get('RMSE', 'N/A')}")
        print(f"   R²: {best_model[1].get('R2', 'N/A')}")
        
        # Calculate ensemble improvement
        if ensemble_rmse < best_model[1].get('RMSE', float('inf')):
            improvement = ((best_model[1].get('RMSE', 0) - ensemble_rmse) / best_model[1].get('RMSE', 1)) * 100
            print(f"   Ensemble Improvement: +{improvement:.2f}%")
        else:
            print(f"   Note: Individual model outperforms ensemble")
    
    # Directional accuracy
    if len(test_targets) > 1:
        true_directions = np.diff(test_targets) > 0
        pred_directions = np.diff(ensemble_predictions) > 0
        directional_accuracy = np.mean(true_directions == pred_directions) * 100
        print(f"\n🎯 DIRECTIONAL ACCURACY: {directional_accuracy:.2f}%")
    
    # Model weights summary
    if hasattr(predictor, 'weights') and predictor.weights:
        print(f"\n⚖️ ENSEMBLE WEIGHTS:")
        for model_name, weight in predictor.weights.items():
            print(f"   {model_name}: {weight:.4f}")
    
    print("\n" + "="*60)
    print("🎨 VISUALIZATION & REPORTING COMPLETE")
    print("="*60)
    print("📊 Generated comprehensive analysis including:")
    print("  ✅ Model performance dashboard")
    print("  ✅ Prediction analysis suite")
    print("  ✅ Feature importance analysis")
    print("  ✅ Comprehensive HTML reports")
    print("  ✅ Executive summary")
    print("="*60)
    
    # =====================================
    # 💾 Enhanced Results Saving
    # =====================================
    
    # Save enhanced results with comprehensive metadata
    try:
        enhanced_results = {
            'ensemble_predictions': ensemble_predictions.tolist(),
            'actual_values': test_targets.tolist(),
            'individual_predictions': {
                model_name: pred[-len(test_targets):].tolist() if len(pred) >= len(test_targets) else pred.tolist()
                for model_name, pred in predictor.predictions.items()
            },
            'performance_metrics': {
                'ensemble': {
                    'MSE': float(ensemble_mse),
                    'RMSE': float(ensemble_rmse),
                    'MAE': float(ensemble_mae),
                    'R2': float(ensemble_r2),
                    'MAPE': float(ensemble_mape)
                },
                'individual_models': individual_metrics
            },
            'model_weights': dict(predictor.weights) if hasattr(predictor, 'weights') else {},
            'generation_timestamp': pd.Timestamp.now().isoformat(),
            'data_points': len(test_targets),
            'models_trained': len(predictor.models)
        }
        
        # Save enhanced results with safe file management
        enhanced_results_save = model_manager.save_model_results(
            enhanced_results,
            "comprehensive_ml_results.json",
            metadata={
                "result_type": "comprehensive_ml_analysis",
                "models_count": len(predictor.models),
                "ensemble_r2": float(ensemble_r2),
                "best_individual_model": best_model[0] if 'best_model' in locals() else 'unknown'
            }
        )
        
        if enhanced_results_save.success:
            print(f"✅ Enhanced results saved: {enhanced_results_save.final_path}")
        else:
            print(f"⚠️ Enhanced results save failed: {enhanced_results_save.error_message}")
            
    except Exception as e:
        print(f"⚠️ Enhanced results saving failed: {str(e)}")
    
    print(f"\n🎉 COMPLETE ML PIPELINE WITH COMPREHENSIVE VISUALIZATION FINISHED!")
    
except Exception as e:
    print(f"❌ Error in visualization and reporting pipeline: {str(e)}")
    print("📊 The basic model training may still have completed successfully")
    print("🔍 Check individual model outputs above for results")